#### Sequence motifs
Understand how to scan DNA for binding sites using CNNs. 

You'll be using a 1D CNN architecture. Pytorch expects the input to each `Conv1d` layer to be in the format `[N C L]`. The values from left to right are:  
`[batch size, number of channels, Length of the input]`

**Data Prep.** Write a function to generate random "DNA" strings and one-hot encode them into tensors.

In [1]:
import numpy as np
import random as rd

NUCL_LIST = ['A', 'T', 'G', 'C'] # nucleotide list
REVC_DIC = {'A':'T', 'T':'A', 'C':'G', 'G': 'C', 'N':'N'}

def revcomp(nucl): # take a nucleotide and return its reverse complement
    revc = ''
    for i in nucl:
        revc += REVC_DIC[i]
    return revc


def gen_dna_string(l, n_prob = 0.1): # generate a random string of dna of length `l`, and return the canonical version
                                     # and probability of `n_prob` for a N in the sequence (default n_prob = 0.1)
    cur_dna = ''
    while len(cur_dna) < l: 
        if rd.random() > (1 - n_prob):
            cur_dna += 'N'
        else:
            cur_dna += rd.choice(NUCL_LIST)
    
    revc = revcomp(cur_dna)
    if revc > cur_dna:
        return cur_dna
    else:
        return revc

In [2]:
# check if the above function works
gen_dna_string(25)

'CCCGTCGCTTAGGGATNCCCATGTA'

### One-hot encode dna strings

scikit-learn's OneHotEncoder is data-driven, so you don't provide it the vocab, just the data, and it 'figures out'

In [3]:
from sklearn.preprocessing import OneHotEncoder 
enc = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
sample_sequences = ['ATGCN', 'AATAGGAGATAC', 'ATGCAGNATT', 'TTAGGG', 'CAGCCAGA']
# tokenize your sequences
tokenized_sequences = [list(seq) for seq in sample_sequences]
# reshape for onehotencoder
all_bases = np.array([base for seq in tokenized_sequences for base in seq]).reshape(-1,1)
enc.fit(all_bases)
print("One hot encoder model trained")

One hot encoder model trained


### Define a function to take a dna string, and return the One-Hot Encoded version

In [4]:
def dna2onehot(nucl):
    x = np.array(list(nucl)).reshape(-1,1)
    o = enc.transform(x)
    return o

In [25]:
# sample usage: 
x = gen_dna_string(10)
o = dna2onehot(x)

#### Simple 1D cnn class
Remember to work with dataloaders, and also use `nn.Sequential`

In [89]:
import torch.nn as nn
import torch

class PrintLayer(nn.Module):
    def forward(self, x):
        print(f"Input shape: {x.shape}")
        return x


motif_finder = nn.Sequential (
    nn.Conv1d(in_channels=5, out_channels=20, kernel_size=8),
    nn.GELU(), 
    nn.MaxPool1d(4),
    nn.Conv1d(20, 40, 8),
    nn.GELU(),
    nn.Flatten(), 
    PrintLayer(),
    # linear layers
    nn.Linear(4760, 512),
    nn.GELU(),
    nn.Dropout(p=0.2),
    nn.Linear(512, 64),
    nn.GELU(),
    nn.Dropout(p=0.2),
    nn.Linear(64, 8),
    nn.GELU(),
    nn.Dropout(p=0.2),
    nn.Linear(8, 2),
)

#### Generate some data for testing

In [90]:
data_size = 500

arr = []
for i in range(data_size):
    x = gen_dna_string(512)
    o = dna2onehot(x)
    arr.append(o)

arr = np.array(arr)

# this is your dataset to check if the model working right :D
X = torch.from_numpy(arr).float()

In [ ]:
x = torch.permute(X, (0, 2, 1))

a = motif_finder(x) # works!

Input shape: torch.Size([500, 4760])
